In [1]:
import cv2
import open3d as o3d
import numpy as np
import colorsys
import pyransac3d as pyrsc

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def visualizer(cloud):
    o3d.visualization.draw_geometries([cloud],
                                  zoom=0.5,
                                  front=[1, 0, 0],
                                  lookat=[0, 0, 0],
                                  up=[0, 0, 1])


In [3]:
pcd = o3d.io.read_point_cloud("3dmodels/Strawberry_Only.ply")

visualizer(pcd)

In [4]:
# -2 to 2
bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=(-1, -1, -1),
                                                    max_bound=(1, 1, 1))

filtered_cloud = pcd.crop(bounding_box)

visualizer(filtered_cloud)

In [86]:
s = 3  #no. of samples used every time to create the shape
p = 0.99 #99%
e = 0.5 # 50%
k = np.log(1-p)/(np.log(1 - np.power((1-e), s)))
print(k)
# k => number of iterations

34.48754705148163


In [47]:

# inlier_cloud = filtered_point_cloud.select_by_index(best_inliers_sphere)
# outlier_cloud = filtered_point_cloud.select_by_index(best_inliers_sphere, invert=True)

# # outlier_cloud.paint_uniform_color([1, 0, 0])
# # inlier_cloud.paint_uniform_color([0, 1, 0])
# o3d.visualization.draw_geometries([outlier_cloud],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [48]:
def cloud_based_point_index(indexes, inlier=False):
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0, 1, 0])
    if not inlier:
        outlier_cloud = filtered_cloud.select_by_index(indexes, invert=True)

        return outlier_cloud

    else:
        inlier_cloud = filtered_cloud.select_by_index(indexes)
        o3d.visualization.draw_geometries([inlier_cloud],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])
        
        return inlier_cloud


In [15]:
point = pyrsc.Point()  #Using RANSAC
# s = 1
center, best_inliers_point = point.fit(np.asarray(filtered_cloud.points), thresh=0.7, maxIteration=10)

inlier_cloud_point = filtered_cloud.select_by_index(best_inliers_point)
outlier_cloud_point = filtered_cloud.select_by_index(best_inliers_point, invert=True)

o3d.visualization.draw_geometries([inlier_cloud_point],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])

In [14]:
# sphere = pyrsc.Sphere()
# # s = 4
# center, radius, best_inliers_sphere = sphere.fit(np.asarray(inlier_cloud_point.points), thresh=0.1, maxIteration=75)

# inlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_sphere)
# outlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_sphere, invert=True)

# # outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
# # inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_sphere],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [7]:
# # plane??

# sphere = pyrsc.Plane()
# # s = 3
# equation, best_inliers_plane = sphere.fit(np.asarray(inlier_cloud_point.points), thresh=0.095, maxIteration=40)

# inlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_plane)
# outlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_plane, invert=True)

# # outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
# # inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_sphere],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [12]:
# print('octree division')
# octree = o3d.geometry.Octree(max_depth=15)
# octree.convert_from_point_cloud(outlier_cloud_sphere, size_expand=0.0001)
# o3d.visualization.draw_geometries([octree])

In [18]:
pcd = inlier_cloud_point

In [19]:
def convert_rgb_2_hsv(all_rgb_colors):
    all_hsv_colors = []

    for i in range(len(all_rgb_colors)):
        temp_color = all_rgb_colors[i]
        temp = colorsys.rgb_to_hsv(temp_color[0], temp_color[1], temp_color[2])
        all_hsv_colors.append(temp)

    all_hsv_colors = np.asarray(all_hsv_colors)

    return all_hsv_colors

all_rgb_colors = np.asarray(pcd.colors)
all_hsv_colors = convert_rgb_2_hsv(all_rgb_colors)


In [20]:
def hsv_filter(hsv_color):
    low_h = 2/360
    high_h = 180/360

    low_s = 0/255
    high_s = 40/255

    low_v = 0/255
    high_v = 255/255

    # removes the background

    if low_h <= hsv_color[0] <= high_h and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return True
    else:
        return False
    

In [21]:
def hsv_filter_red(hsv_color):

    # print("color:", hsv_color)
    low_h = 0/360
    high_h = 40/360

    low_h_1 = 340/360
    high_h_1 = 360/360

    # 330 to 30 degrees

    low_s = 0.1
    high_s = 1

    low_v = 0.4
    high_v = 1

    # red
    # Lower bound: (0, 100, 100)
    # Upper bound: (10, 255, 255)

    if ((low_h <= hsv_color[0] <= high_h) or (low_h_1 <= hsv_color[0] <= high_h_1))and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return True
    else:
        return False

    # red = hsv_color[0]
    # green = hsv_color[1]
    # blue = hsv_color[2]

    # print(red)


    if red >= (150/255) and red <= (250/255):
        return True
    else:
        return False
    

In [22]:
new_points = []
new_colors = []
new_colors_RGB_u = []

for index, new_color in enumerate(all_hsv_colors):
    new_point = pcd.points[index]
    new_color_RGB = all_rgb_colors[index]
    # if hsv_filter(new_color):
    #     new_colors_RGB_u.append(new_color_RGB)
    #     new_points.append(new_point)
    #     new_colors.append(new_color)
    #     # print(new_color_RGB)
    if hsv_filter_red(new_color):
        # break
        new_colors_RGB_u.append(new_color_RGB)
        new_points.append(new_point)
        new_colors.append(new_color)

new_colors_RGB_u = np.asarray(new_colors_RGB_u)
new_colors = np.asarray(new_colors)
new_points = np.asarray(new_points)

filtered_point_cloud = o3d.geometry.PointCloud()
filtered_point_cloud.points = o3d.utility.Vector3dVector(new_points)
filtered_point_cloud.colors = o3d.utility.Vector3dVector(new_colors_RGB_u)
visualizer(filtered_point_cloud)

In [32]:
cl, ind = filtered_point_cloud.remove_statistical_outlier(nb_neighbors=70,
                                                    std_ratio=1.0)
cl.estimate_normals()
visualizer(cl)


In [33]:
# plane??

sphere = pyrsc.Plane()
# s = 3
equation, best_inliers_plane = sphere.fit(np.asarray(cl.points), thresh=0.095, maxIteration=40)

inlier_cloud_plane = cl.select_by_index(best_inliers_plane)
outlier_cloud_plane = cl.select_by_index(best_inliers_plane, invert=True)

# outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# inlier_cloud_plane.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([outlier_cloud_plane],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])

In [28]:
# ply_point_cloud = o3d.data.PLYPointCloud()
# pcd = o3d.io.read_point_cloud(ply_point_cloud.path)

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        cl.cluster_dbscan(eps=0.02, min_points=30, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
# colors = cl.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
# colors[labels < 0] = 0
cl.colors = o3d.utility.Vector3dVector(all_rgb_colors[:, :3])
visualizer(cl)

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 59
point cloud has 59 clusters
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [166]:
import open3d as o3d
from sklearn.cluster import DBSCAN
import numpy as np

def cluster_dbscan(point_cloud, eps, min_samples):
    # Convert Open3D point cloud to numpy array
    points = np.asarray(point_cloud.points)

    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(points)

    # Get unique cluster labels
    unique_labels = np.unique(labels)

    # Calculate cluster sizes
    cluster_sizes = {label: np.sum(labels == label) for label in unique_labels}

    return labels, cluster_sizes

def remove_small_clusters(point_cloud, labels, min_cluster_size):
    # Convert Open3D point cloud to numpy array
    points = np.asarray(point_cloud.points)
    colors = np.asarray(point_cloud.colors)

    # Get unique cluster labels
    unique_labels, cluster_counts = np.unique(labels, return_counts=True)

    # Filter out small clusters
    large_clusters_mask = cluster_counts >= min_cluster_size
    large_clusters = unique_labels[large_clusters_mask]

    # Filter points and colors belonging to large clusters
    filtered_points = points[np.isin(labels, large_clusters)]
    filtered_colors = colors[np.isin(labels, large_clusters)]

    # Create a new point cloud with filtered points
    filtered_point_cloud_new = o3d.geometry.PointCloud()
    filtered_point_cloud_new.points = o3d.utility.Vector3dVector(filtered_points)
    filtered_point_cloud_new.colors = o3d.utility.Vector3dVector(filtered_colors)

    return filtered_point_cloud_new


# Example usage:
# Assuming 'point_cloud' is your point cloud data
eps = 0.1  # DBSCAN epsilon parameter
min_samples = 10  # Minimum number of samples required to form a cluster
min_cluster_size = 10  # Minimum size of a cluster to keep

# Perform DBSCAN clustering
labels, cluster_sizes = cluster_dbscan(filtered_point_cloud, eps, min_samples)

# Remove small clusters
# filtered_point_cloud = remove_small_clusters(cl, labels, min_cluster_size)

# Visualize the filtered point cloud
# visualizer(filtered_point_cloud)

In [167]:
color = [
    [255, 0, 0],   # Red
    [0, 255, 0],   # Green
    [0, 0, 255],   # Blue
    [255, 255, 0], # Yellow
    [255, 0, 255], # Magenta
    [0, 255, 255], # Cyan
    [128, 0, 0],   # Maroon
    [0, 128, 0],   # Green (dark)
    [0, 0, 128],   # Navy
    [128, 128, 0],  # Olive
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 128],# Gray
    [255, 165, 0],  # Orange
    [255, 192, 203],# Pink
    [0, 0, 0],      # Black
    [255, 255, 255],# White
    [128, 0, 0],    # Maroon (dark)
    [0, 128, 0],    # Green (medium)
    [0, 0, 128]  
]

outlier_cloud_sph = []

for i in range(len(cluster_sizes)):
    # temp_p = np.asarray(pcd.points)[kmeans.labels_ == i]
    temp_p = np.where(labels == i)
    # print(temp_p[0])
    temp_sphere = cl.select_by_index(temp_p[0])
    temp_sphere.paint_uniform_color(color[i])
    outlier_cloud_sph.append(temp_sphere)

# visualizer(filtered_point_cloud)

geometries_to_visualize = [outlier_cloud_sph[i] for i in range(len(cluster_sizes))]  # Adjust the range as needed

o3d.visualization.draw_geometries(geometries_to_visualize,
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to

In [168]:
from sklearn.cluster import KMeans

# filtered_point_cloud = o3d.geometry.PointCloud()
# def filter_red(points, colors):
# Convert RGB colors to feature space suitable for clustering
features = np.array(filtered_point_cloud.points)  # Use RGB directly as features

# Apply K-means clustering
kmeans = KMeans(n_clusters=5, random_state=0).fit(features)

# Identify cluster representing red points
# kmeans.cluster_centers_ -> gives the cenetrs of the clusters so i.e of the colors
red_cluster_index = np.argmin(np.mean(kmeans.cluster_centers_, axis=1))

# Filter points based on cluster membership
# red_points = np.asarray(pcd.points)[kmeans.labels_ == red_cluster_index]
# red_colors = np.asarray(pcd.colors)[kmeans.labels_ == red_cluster_index]

# return red_points, red_colors

# filtered_point_cloud.points = o3d.utility.Vector3dVector(red_points)
# filtered_point_cloud.colors = o3d.utility.Vector3dVector(red_colors)

color = [
    [255, 0, 0],   # Red
    [0, 255, 0],   # Green
    [0, 0, 255],   # Blue
    [255, 255, 0], # Yellow
    [255, 0, 255], # Magenta
    [0, 255, 255], # Cyan
    [128, 0, 0],   # Maroon
    [0, 128, 0],   # Green (dark)
    [0, 0, 128],   # Navy
    [128, 128, 0],  # Olive
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 128],# Gray
    [255, 165, 0],  # Orange
    [255, 192, 203],# Pink
    [0, 0, 0],      # Black
    [255, 255, 255],# White
    [128, 0, 0],    # Maroon (dark)
    [0, 128, 0],    # Green (medium)
    [0, 0, 128]  
]

outlier_cloud_sph = []

for i in range(len(kmeans.cluster_centers_)):
    # temp_p = np.asarray(pcd.points)[kmeans.labels_ == i]
    temp_p = np.where(kmeans.labels_ == i)
    # print(temp_p[0])
    temp_sphere = pcd.select_by_index(temp_p[0])
    temp_sphere.paint_uniform_color(color[i])
    outlier_cloud_sph.append(temp_sphere)

# visualizer(filtered_point_cloud)

geometries_to_visualize = [outlier_cloud_sph[i] for i in range(len(kmeans.cluster_centers_))]  # Adjust the range as needed

o3d.visualization.draw_geometries(geometries_to_visualize,
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])



[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [39]:
print(kmeans.cluster_centers_)
print(np.mean(kmeans.cluster_centers_, axis=1))

[[0.11900472 0.15205582 0.08048408]
 [0.48148465 0.61985623 0.41657004]
 [0.29789122 0.34944458 0.21945515]
 [0.68862662 0.8572229  0.67027523]
 [0.35766358 0.55965098 0.10678572]]
[0.11718154 0.50597031 0.28893032 0.73870825 0.34136676]


In [41]:
new_points = []
new_colors = []
new_colors_RGB_u = []

for index, new_color in enumerate(all_hsv_colors):
    new_point = pcd.points[index]
    new_color_RGB = all_rgb_colors[index]
    # if hsv_filter(new_color):
    #     new_colors_RGB_u.append(new_color_RGB)
    #     new_points.append(new_point)
    #     new_colors.append(new_color)
        # print(new_color_RGB)
    if hsv_filter_red(new_color):
        # break
        new_colors_RGB_u.append(new_color_RGB)
        new_points.append(new_point)
        new_colors.append(new_color)

new_colors_RGB_u = np.asarray(new_colors_RGB_u)
new_colors = np.asarray(new_colors)
new_points = np.asarray(new_points)



In [61]:
filtered_point_cloud = o3d.geometry.PointCloud()
filtered_point_cloud.points = o3d.utility.Vector3dVector(new_points)
filtered_point_cloud.colors = o3d.utility.Vector3dVector(new_colors_RGB_u)
visualizer(filtered_point_cloud)

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [202]:
# radii = [0.005, 0.01]
# outlier_cloud_sphere.estimate_normals()
# rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     outlier_cloud_sphere, o3d.utility.DoubleVector(radii))

# visualizer(rec_mesh)

# # Can make much better meshes


In [211]:
inlier_cloud_point.estimate_normals()


In [212]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        inlier_cloud_point, depth=15)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

[Open3D DEBUG] Input Points / Samples: 100723 / 100718
[Open3D DEBUG] #   Got kernel density: 2.064 (s), 4297.86 (MB) / 19821.9 (MB) / 19843 (MB)
[Open3D DEBUG] #     Got normal field: 0.483 (s), 4280.66 (MB) / 19821.9 (MB) / 19843 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 8.197693e-06 / 8.256962e-01
[Open3D DEBUG] #       Finalized tree: 2.78 (s), 4868.52 (MB) / 19821.9 (MB) / 19843 (MB)
[Open3D DEBUG] #  Set FEM constraints: 32.9 (s), 4310.96 (MB) / 19821.9 (MB) / 19843 (MB)
[Open3D DEBUG] #Set point constraints: 0.764 (s), 4305.55 (MB) / 19821.9 (MB) / 19843 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 20237785 / 1433592 / 21695305
[Open3D DEBUG] Memory Usage: 4305.551 MB
[Open3D DEBUG] # Linear system solved: 27.892 (s), 4368.53 (MB) / 19821.9 (MB) / 19843 (MB)
[Open3D DEBUG] Got average: 0.807000 (s), 4329.9 (MB) / 19821.9 (MB) / 19843 (MB)
[Open3D DEBUG] Iso-Value: 4.999772e-01 = 5.035921e+04 / 1.007230e+05
[Open3D DEBUG] #          Total Solve:     108.7

In [213]:
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [128]:
# Visualize the first point cloud
o3d.visualization.draw_geometries([filtered_point_cloud], window_name='Filtered Point Cloud')

# o3d.io.write_point_cloud("saved_point_cloud.ply", filtered_point_cloud)
# o3d.io.write_point_cloud("outlier_cloud_sphere.ply", outlier_cloud_sphere)


# Visualize the second point cloud
# o3d.visualization.draw_geometries([outlier_cloud_sphere], window_name='Outlier Cloud Sphere')